# Cálculos de aproximación de esfuerzos en la suspensión

# Cálculos de esfuerzos en suspensión

In [1]:
import numpy             as np
import matplotlib.pyplot as plt

## Planteamiento de torques

Se definirá la fuerza de tracción que produce el neumático como el vector $ C $.
$ C $ se divide en sus dos componentes ortogonales y se definirá $ N $ como la componente normal de la interacción y $ G $ como la tracción lateral.

De los dos nodos que representan los extremos de la mangueta parten las fuerzas $ A $ y $ B $. Estas se usarán para calcular los esfuerzos en la malla de la suspensión.

Se tomará el nodo correspondiente al enlace inferior de la suspensión en el chasis como el origen del análisis.

Partiendo de este origen, los vectores $A$ y $B$ se obtienen de la siguiente manera:

$$ A = \begin{bmatrix} G\frac{b_y-c_y}{a_y-b_y} \\ N\frac{b_x-c_x}{a_x-b_x} \end{bmatrix} $$
$$ B = \begin{bmatrix} G\frac{a_y-c_y}{b_y-a_y} \\ N\frac{a_x-c_x}{b_x-a_x} \end{bmatrix} $$

In [2]:
dist_to_C = np.array([-399.18, -158.58])
dist_to_A = np.array([-329.48,  216.58])
dist_to_B = np.array([-355.53,   -1.87])
dist_to_E = np.array([- 18.80,  207.03])

G         = np.array([   0.00,  441.30])

def get_A_B(grip, dist_A, dist_B, dist_C):
	A = np.array([
		grip[0]*(dist_B[1]-dist_C[1])/(dist_A[1]-dist_B[1]),
		grip[1]*(dist_B[0]-dist_C[0])/(dist_A[0]-dist_B[0])
	])
	B = np.array([
		grip[0]*(dist_A[1]-dist_C[1])/(dist_B[1]-dist_A[1]),
		grip[1]*(dist_A[0]-dist_C[0])/(dist_B[0]-dist_A[0])
	])

	return A, B

def angle_vect(vec):
	return np.arctan2(vec[1], vec[0])

def get_tension(force_a, force_b, axis_vect):
	axis_angle = angle_vect(axis_vect)
	norm_a = np.linalg.norm(force_a)
	norm_b = np.linalg.norm(force_b)

	angle_a = angle_vect(force_a)
	angle_b = angle_vect(force_b)

	return norm_b*np.cos(angle_b-axis_angle) - norm_a*np.cos(angle_a-axis_angle)

def to_vect(magnitude, angle):
	return np.array([
		magnitude*np.cos(angle),
		magnitude*np.sin(angle)
	])

## Nodo libre

El nodo libre es aquel que tiene enlazado el A-arm que no posee ni push-rod ni pull-rod, por lo que solo experimenta las fuerzas de tensión/compresión de las barras.

En este caso tenemos que $ \alpha + \beta = A $.

Para obtener la magnitud de los vectores tenemos:
$$ [\beta] = [A]\frac{sin[ang(A)]-cos[ang(A)]tg[ang(\alpha)]}
                     {sin[ang(\beta)]-cos[ang(\beta)]tg[ang(\alpha)]} $$
$$ [\alpha] =  \frac{[A]cos[ang(A)]-[\beta]cos[ang(\beta)]}{cos[ang(\alpha)]} = \frac{[A]sin[angle(A)]-[\beta]sin[ang(\beta)]}{sin[ang(\alpha)]} $$

In [3]:
A, B = get_A_B(G, dist_to_A, dist_to_B, dist_to_C)

alpha_beam = dist_to_B - dist_to_A
betha_beam = dist_to_E - dist_to_A

betha_norm = np.linalg.norm(A) * (
	(np.sin(angle_vect(A))) - np.cos(angle_vect(A))*np.tan(angle_vect(alpha_beam)) /
	(np.sin(angle_vect(betha_beam) - np.cos(angle_vect(betha_beam)))*np.tan(angle_vect(alpha_beam)))
)
betha = to_vect(betha_norm, angle_vect(dist_to_E-dist_to_A))

alpha = A - betha

# alpha_tension = get_tension(A, B, dist_to_A-dist_to_B)
# alpha         = to_vect(alpha_tension, angle_vect(dist_to_A-dist_to_B))

print( '====Primary node force vectors====')
print(f'A: {A}')
print(f'B: {B}')
# print(f'{G*np.flip(dist_to_C) + A*np.flip(dist_to_A) + B*np.flip(dist_to_B)}')
print( '===========Beam vectors===========')
print(f'Alpha: {alpha_beam}')
print(f'Betha: {betha_beam}')
print( '=====Upper beam force vectors=====')
print(f'Alpha: {alpha}')
print(f'Betha: {betha}')
print( '======Upper beam force norms======')
print(f'Betha: {betha_norm}')
# print(alpha_tension)

====Primary node force vectors====
A: [  0.         739.45278311]
B: [   -0.         -1180.75278311]
===========Beam vectors===========
Alpha: [ -26.05 -218.45]
Betha: [310.68  -9.55]
=====Upper beam force vectors=====
Alpha: [-739.1036806   762.17210894]
Betha: [739.1036806  -22.71932583]
======Upper beam force norms======
Betha: 739.4527831094069


In [4]:
# betha_ = A - alpha

angle_by_force = angle_vect(betha)      % (np.pi)
angle_by_geome = angle_vect(betha_beam) % (np.pi)

print('==========Betha==========')
print(betha)
print(betha_beam)
print(np.degrees(angle_by_force))
print(np.degrees(angle_by_geome))
if abs(angle_by_force - angle_by_geome) > 1e-4:
	print('FUCK!')

angle_by_force = angle_vect(alpha)      % (np.pi)
angle_by_geome = angle_vect(alpha_beam) % (np.pi)

print('==========Alpha==========')
print(alpha)
print(alpha_beam)
print(np.degrees(angle_by_force))
print(np.degrees(angle_by_geome))
if abs(angle_by_force - angle_by_geome) > 1e-4:
	print('FUCK!')


==========Betha==========
[739.1036806  -22.71932583]
[310.68  -9.55]
178.2393380569692
178.2393380569692
==========Alpha==========
[-739.1036806   762.17210894]
[ -26.05 -218.45]
134.1196690284846
83.19963418622574
FUCK!
